# Grown Intern Task 2

## Project Title: Gender & Age Detection

### Installing Libararies

- First we have to isnatll cmake to further install packages for face detectiona nd age detection

In [58]:
!pip install cmake

!pip install --upgrade pip
!pip install --upgrade setuptools


In [21]:
!pip install "C:\Users\PMLS\Downloads\dlib-19.24.99-cp312-cp312-win_amd64.whl"

Processing c:\users\pmls\downloads\dlib-19.24.99-cp312-cp312-win_amd64.whl


In [23]:
!pip install dlib

### Implementation

In [77]:
import cv2  # for computer vision task 
import dlib # contains machine learning algorithims like face detection
import numpy as np
import os # Library for interacting with the operating system, like handling file paths.

faceproto="opencv_face_detector.pbtxt" # models configuration fro face detection
facemode1="opencv_face_detector_unit8.pb" # Model weights file for face detection

age_weights = "age_net.caffemodel" # weights to age detection model files
age_config = "age_deploy.prototxt" #configuration file for age detection model
age_Net = cv2.dnn.readNet(age_config, age_weights) #loading the iamge detection model

gender_weights = "gender_net.caffemodel" # paths to gender detection model files
gender_config = "gender_deploy.prototxt" #weights for model

gender_Net = cv2.dnn.readNet(gender_config, gender_weights) #loading the model 
 
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)',  # age and gender labels and model mean values
           '(25-32)', '(38-43)', '(48-53)','(54-60)','(60-80)','(80-100)']
genderList = ['Male', 'Female']
model_mean = (78.4263377603, 87.7689143744, 114.895847746) #mean values for image processing

face_detector = dlib.get_frontal_face_detector() # model for face detection
                                             # directory containing images ( in my device)
image_dir = r"C:\Users\PMLS\Desktop\pics"  # adjust this path if necessary

# loop through all images in the directory, to detect them one by one
for filename in os.listdir(image_dir):
    if filename.lower().endswith((".jpeg", ".jpg", ".png")):
        img_path = os.path.join(image_dir, filename)
        
        img = cv2.imread(img_path) # loading the image
        frame = img.copy()

        img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # converting to grayscale
        
        faces = face_detector(img_gray) # detecting faces
        
        if not faces:  # if no faces are detected
            mssg = 'No face detected'
            cv2.putText(img, f'{mssg}', (40, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (200), 2)
            cv2.imshow('Age and Gender detected', img)
            cv2.waitKey(0)
        else:
            Boxes = []  # to store the face coordinates
            mssg = 'Face Detected'  # to display on image
            
            # Bounding face
            for face in faces:
                x = face.left()  # extracting the face coordinates
                y = face.top()
                x2 = face.right()
                y2 = face.bottom()

                # rescaling those coordinates for our image
                box = [x, y, x2, y2]
                Boxes.append(box)
                cv2.rectangle(frame, (x, y), (x2, y2),
                              (0, 200, 200), 2)  #storing  these coordinates in a list and draw a rectangle around the face on the image.

            for box in Boxes:
                face = frame[box[1]:box[3], box[0]:box[2]]  #oordinates of the bounding box around the detected face.

                blob = cv2.dnn.blobFromImage( # this function converts the image into a format suitable for neural network input
                    face, 1.0, (227, 227), model_mean, swapRB=False)

                age_Net.setInput(blob)  #  sets the preprocessed face blob as the input for the age detection neural network (age_Net).
                age_preds = age_Net.forward() # runs a forward pass through the neural network to get the age prediction.
                age = ageList[age_preds[0].argmax()] #inds the index of the highest probability in the predictions, indicating the most likely age group.

                gender_Net.setInput(blob)  # gender prediction
                gender_preds = gender_Net.forward()
                gender = genderList[gender_preds[0].argmax()]

                cv2.putText(frame, f'{mssg}: {age}, {gender}', (box[0],
                                                                box[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, # represeny the  font size, font style , color, and text size showing above image 
                             (255, 255, 255), 1, cv2.LINE_AA)

            cv2.imshow("Detecting Age and Gender", frame) #displays the image with the age and gender predictions.
            cv2.waitKey(0) # allows you the watch the image as long as you want 

cv2.destroyAllWindows() # closes open cv after the directory has  looped over all images
